# Relazione Finale - Gruppo Dig Data
Componenti Gruppo - Alexandru Pavel, Simone Garzarella

# Indice
TODO: Mettere i numeri sezioni solo qui per chiarezza ulteriore (anche se non serve)
- [Introduzione](#introduction)
    *  [Descrizione Problema](#problem-description)
    *  [Specifiche Software](#sw-specs)
    *  [Specifiche Software](#hw-specs)
- [Analisi Dataset](#data-analysis)
    * [Historical Stock Prices](#hsp)
    * [Historical Stocks](#hs)
- [Job 1](#job1)
    * [MapReduce](#mapreduce1)
    * [Hive](#hive1)
    * [Spark](#spark1)
- [Job 2](#job2)
    * [MapReduce](#mapreduce2)
    * [Hive](#hive2)
    * [Spark](#spark2)
- [Job 3](#job3)
    * [MapReduce](#mapreduce3)
    * [Hive](#hive3)
    * [Spark](#spark3)
- [Grafici Finali](#plots)
    * [Tipo1](#plot1)
    * [Tipo2](#plot2)
    * [Tipo3](#plot3)
- [Conclusioni](#conclusions)

# Introduzione <a name="introduction"></a>
Some introduction text, formatted in heading 2 style

## Descrizione Problema <a name="problem-description"></a>
This is a sub paragraph, formatted in heading 3 style

## Specifiche Software <a name="sw-specs"></a>
This is a sub paragraph, formatted in heading 3 style

## Specifiche Hardware <a name="hw-specs"></a>
This is a sub paragraph, formatted in heading 3 style

# Analisi Dataset <a name="data-analysis"></a>
The first paragraph text

## Historical Stock Prices <a name="hsp"></a>
This is a sub paragraph, formatted in heading 3 style

## Historical Stocks <a name="hs"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 1 <a name="job1"></a>
The first paragraph text

## MapReduce <a name="mapreduce1"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive1"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="spark1"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 2 <a name="job2"></a>
The first paragraph text

## MapReduce <a name="mapreduce2"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive2"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="spark2"></a>
This is a sub paragraph, formatted in heading 3 style

# Job 3 <a name="job3"></a>
The first paragraph text

## MapReduce <a name="mapreduce3"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="hive3"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="spark3"></a>
This is a sub paragraph, formatted in heading 3 style

# Grafici <a name="plots"></a>
The first paragraph text

## MapReduce <a name="plot1"></a>
This is a sub paragraph, formatted in heading 3 style

## Hive <a name="plot2"></a>
This is a sub paragraph, formatted in heading 3 style

## Spark <a name="plot3"></a>
This is a sub paragraph, formatted in heading 3 style

# Conclusioni <a name="conclusions"></a>
The first paragraph text
